In [1]:
import time
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df = pd.read_csv(r"C:\Users\nishi\code\Git\California Housing Prices\California Housing Prices\housing.csv\housing.csv")
df.tail()
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
enc = OrdinalEncoder()
ocean_proximity=['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']
df['ocean_proximity']=enc.fit_transform(df[['ocean_proximity']])
df = df.fillna(df.mean())
cols_to_use=["longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income"]
poly = PolynomialFeatures(degree=2,include_bias=False)

X_poly=poly.fit_transform(df[cols_to_use])
poly_features=pd.DataFrame(X_poly,columns=poly.get_feature_names_out(cols_to_use),index=df.index)
df=df.drop(cols_to_use,axis=1)
df=pd.concat((df,poly_features),axis=1)
df.head()

,median_house_value,ocean_proximity,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,...,total_bedrooms^2,total_bedrooms population,total_bedrooms households,total_bedrooms median_income,population^2,population households,population median_income,households^2,households median_income,median_income^2
0,452600.0,3.0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,...,16641.0,41538.0,16254.0,1073.9508,103684.0,40572.0,2680.7144,15876.0,1048.9752,69.308955
1,358500.0,3.0,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,...,1223236.0,2655506.0,1258628.0,9181.3484,5764801.0,2732338.0,19931.6614,1295044.0,9446.9932,68.913242
2,352100.0,3.0,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,...,36100.0,94240.0,33630.0,1378.9060,246016.0,87792.0,3599.6704,31329.0,1284.5598,52.669855
3,341300.0,3.0,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,...,55225.0,131130.0,51465.0,1326.1285,311364.0,122202.0,3148.8498,47961.0,1235.8389,31.844578
4,342200.0,3.0,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,...,78400.0,158200.0,72520.0,1076.9360,319225.0,146335.0,2173.1030,67081.0,996.1658,14.793254


In [4]:
X = df.drop(columns=['median_house_value']).values
y = df['median_house_value'].values

In [5]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2,random_state=47)

In [6]:
scaler_x=StandardScaler()
scaler_y=StandardScaler()
scaler_x.fit(X_train)
X_train = scaler_x.transform(X_train)
X_test  = scaler_x.transform(X_test)
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)
scaler_y.fit(y_train)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)
X_train=torch.tensor(X_train,dtype=torch.float32)
y_train=torch.tensor(y_train,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
y_test=torch.tensor(y_test,dtype=torch.float32)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
X_train=X_train.to(device)
y_train=y_train.to(device)
X_test=X_test.to(device)
y_test=y_test.to(device)

In [8]:
train_loader=DataLoader(TensorDataset(X_train,y_train),batch_size=64,shuffle=True)
test_loader=DataLoader(TensorDataset(X_test,y_test),batch_size=32)

In [9]:
class neural(torch.nn.Module):
    def __init__(self, num_features,num_hidden_1,num_hidden_2,num_hidden_3):
        super().__init__()
        self.my_network = torch.nn.Sequential(
            
            torch.nn.Flatten(),
            torch.nn.Linear(num_features, num_hidden_1),
            torch.nn.LeakyReLU(),
    
            torch.nn.Linear(num_hidden_1, num_hidden_2),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.3),

            torch.nn.Linear(num_hidden_2, num_hidden_3),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(num_hidden_3, 1))
    
    def forward(self,x):
        targets=self.my_network(x)
        return targets
            

In [10]:
def train(model,num_epochs,train_data,test_data,optimizer,device):
    minibatch_loss=[]
    start_time=time.time()
    for epoch in range(num_epochs):
        
        model.train()

        for batch_idx, (features,targets) in enumerate(train_data):
            features=features.to(device)
            targets=targets.to(device)

            y_hat=model(features)
            loss=torch.mean((y_hat-targets)**2)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            minibatch_loss.append(loss.item())
            
            if not batch_idx % 32:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_data):04d} '
                      f'| Loss: {loss.item():.4f}')
                
        elapsed_time=time.time() - start_time

        avg_loss=torch.tensor(minibatch_loss).mean().item()
        print(f"{'-'*44}\ncost={avg_loss}\nelapsed_time={elapsed_time:.2f}sec")
        
    return minibatch_loss

def test(model,test_data,device):
    model.eval()
    test_loss=[]
    cost=torch.nn.MSELoss()
    with torch.no_grad():
        for (features,targets) in test_data:
            features=features.to(device)
            targets=targets.to(device)

            y_hat=model(features)
            loss=cost(y_hat,targets)
            test_loss.append(loss.item())
    avg_loss=torch.tensor(test_loss).mean().item()
    print(f"cost={avg_loss}")
    return test_loss

In [11]:
model=neural(num_features=X_train.shape[1],num_hidden_1=91,num_hidden_2=91,num_hidden_3=91)
model=model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
minibatch_loss=train(model=model,num_epochs=100
                     ,train_data=train_loader
                     ,test_data=test_loader,optimizer=optimizer
                     ,device=device)

Epoch: 001/100 | Batch 0000/0258 | Loss: 0.9473
Epoch: 001/100 | Batch 0032/0258 | Loss: 0.4153
Epoch: 001/100 | Batch 0064/0258 | Loss: 0.3339
Epoch: 001/100 | Batch 0096/0258 | Loss: 0.4930
Epoch: 001/100 | Batch 0128/0258 | Loss: 0.4185
Epoch: 001/100 | Batch 0160/0258 | Loss: 0.2929
Epoch: 001/100 | Batch 0192/0258 | Loss: 0.1675
Epoch: 001/100 | Batch 0224/0258 | Loss: 0.3330
Epoch: 001/100 | Batch 0256/0258 | Loss: 0.3776
--------------------------------------------
cost=0.3999231159687042
elapsed_time=1.38sec
Epoch: 002/100 | Batch 0000/0258 | Loss: 0.2641
Epoch: 002/100 | Batch 0032/0258 | Loss: 0.2787
Epoch: 002/100 | Batch 0064/0258 | Loss: 0.1414
Epoch: 002/100 | Batch 0096/0258 | Loss: 0.1996
Epoch: 002/100 | Batch 0128/0258 | Loss: 0.2812
Epoch: 002/100 | Batch 0160/0258 | Loss: 0.2692
Epoch: 002/100 | Batch 0192/0258 | Loss: 0.2119
Epoch: 002/100 | Batch 0224/0258 | Loss: 0.2546
Epoch: 002/100 | Batch 0256/0258 | Loss: 0.1947
--------------------------------------------
c

In [12]:
test_loss=test(model=model,test_data=test_loader,device=device)

cost=0.20355196297168732


In [13]:
org_loss=[]
for (features,targets) in test_loader:
    targets=targets.to(device)
    features=features.to(device)
    y_hat=model(features)
    y_hat=scaler_y.inverse_transform(y_hat.detach().cpu().numpy().reshape(-1,1))
    y=scaler_y.inverse_transform(targets.detach().cpu().numpy().reshape(-1,1))
    loss=np.mean((y_hat-y)**2)
    org_loss.append(loss)
loss=np.mean(org_loss)
rms_loss=np.sqrt(loss)
print(f"RMSE={rms_loss}")

RMSE=51908.18359375
